In [ ]:
import pandas as pd 

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
def show_missing_values():
    nulls = data.isnull()
    null_values = nulls.sum().sort_values(ascending=False)
    percentage = nulls.sum() / nulls.count()*100
    missing_values = pd.concat([null_values, percentage], axis=1, keys=['Number', 'Percentage'], sort=False)
    return missing_values[missing_values['Percentage'] != 0]

show_missing_values()

In [ ]:
keepCols = ["Severity", "Start_Time", "Start_Lat", "Start_Lng","City", "State", "Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)"]
newData = data[data.columns.intersection(keepCols)]
newData.head()

In [ ]:
newData.insert(11, "Road_Type", "")

In [ ]:
import requests
def makeWeatherApiRequest(row):
    url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    apiKey = "YZVW87VHVPK8M8W5WXYD4LYR8"
    locationString = str(row["Start_Lat"]) + "," + str(row["Start_Lng"])
    dateString = str(row["Start_Time"].replace(" ", "T0"))[:20]
    requestUrl = url + locationString + "/" + dateString + "?key=" + apiKey
    response = requests.get(requestUrl)
    return response

In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point

firstRow = newData.iloc[0]

lat = firstRow["Start_Lat"]
lon = firstRow["Start_Lng"]
print(lat,lon)

# Download road network as a so-called Graph object
# The network will be centered at your coordinate
# and includes all nodes in the vicinity of 2 km
G = ox.graph_from_point((lat,lon), dist=250, network_type='all')
# plot if you wish
ox.plot_graph(G)

# Convert Graph to graph data frame
gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
# extract roads and some properties
roads = gdf[["geometry","highway"]].values.tolist()
# calculate and attach distance
roads_with_distances = [(road, Point(tuple(reversed((lat,lon)))).distance(road[0])) for road in roads]
# sort by distance
roads_with_distances = sorted(roads_with_distances, key=lambda x: x[1])

# Select closest road
closest_road = roads_with_distances[0]
print(closest_road[0][1])
print(roads)

In [ ]:
def getRoadType(row):
    try:
        lat = row["Start_Lat"]
        lon = row["Start_Lng"]
        try:
            G = ox.graph_from_point((lat,lon), dist=2000, network_type='all')
        except Exception as e:
            return ""
        gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
        roads = gdf[["geometry","highway"]].values.tolist()
        if len(roads) < 1:
            return ""
        roads_with_distances = [(road, Point(tuple(reversed((lat,lon)))).distance(road[0])) for road in roads]
        roads_with_distances = sorted(roads_with_distances, key=lambda x: x[1])

        closest_road = roads_with_distances[0]
        return closest_road[0][1]
    except Exception as e:
        return ""

In [ ]:
#very long time to run
smallData = newData.head(20000)
for index, row in smallData.iterrows():
    smallData.at[index,"Road_Type"]=getRoadType(row)
smallData.to_csv("dataFromNight.csv")
smallData.head()

In [ ]:
#very long time to run
tailData = newData.tail(5000)
for index, row in tailData.iterrows():
    tailData.at[index,"Road_Type"]=getRoadType(row)

tailData.to_csv("tailOfSmallDataFromNight.csv")

In [ ]:
dfBoth = pd.concat([smallData, tailData])
dfBoth.to_csv("concatedData.csv")

In [ ]:
#long time to run
for index, row in dfBoth.iterrows():
    if dfBoth.at[index,"Road_Type"] == "":
        print("Getting road type for index: " + str(index))
        dfBoth.at[index,"Road_Type"]=getRoadType(row)

In [ ]:
dfBoth['Road_Type'] = dfBoth['Road_Type'].astype(str)

In [ ]:
dfBoth.to_csv("finalData.csv")